# 使用TensorFlow默认Image进行推理

需要用到Tensorflow 和Keras , kernel 选择conda_tensorflow_p36

## 把训练好的模型存放S3上
model.tar.gz内目录结构如下
```
model.tar.gz
└── tf_server
    └── 1
        ├── saved_model.pb
        └── variables
            ├── variables.data-00000-of-00001
            └── variables.index
```

## 部署模型到SageMaker

In [ ]:
from sagemaker.tensorflow.model import TensorFlowModel
from sagemaker import get_execution_role

role = get_execution_role()
#role="arn:aws-cn:iam::315505707008:role/service-role/AmazonSageMaker-ExecutionRole-20200430T124235" 

model_uri = "s3://nowfox/data/cat-vs-dog-output/tensorflow-training-2020-08-24-09-33-46-405/output/model.tar.gz"
endpoint_name = "sagemaker-cat-vs-dog-2"
my_model = TensorFlowModel(
            model_data=model_uri, 
            role=role,
            framework_version='1.15.2')

#该步骤大概需要10分钟
my_model.deploy(initial_instance_count=1,
                                endpoint_name=endpoint_name,
                                instance_type='ml.t2.medium'
                                )

## 推理
### 读取数据

In [ ]:
from keras.preprocessing import image
import json
import numpy as np


IMAGE_WIDTH = 150
IMAGE_HEIGHT = 150
# 修改测试图片地址
#image_paths = 'test/cat.681.jpg'
image_paths = 'test/dog.592.jpg'
images = image.load_img(image_paths, target_size=(IMAGE_WIDTH, IMAGE_HEIGHT))
input_image = image.img_to_array(images)
input_image = np.expand_dims(input_image, axis=0)
input_image /= 255.

input_images = input_image.tolist()


image_paths = 'test/cat.681.jpg'
#image_paths = 'test/dog.592.jpg'
images = image.load_img(image_paths, target_size=(IMAGE_WIDTH, IMAGE_HEIGHT))
input_image = image.img_to_array(images)
input_image = np.expand_dims(input_image, axis=0)
input_image /= 255.

input_images = input_image.tolist()

data = {"name": 'tensorflow/serving/predict',"signature_name":'predict',"inputs":input_images}

In [ ]:
import sagemaker
from sagemaker.tensorflow.model import TensorFlowPredictor

endpoint_name = "sagemaker-cat-vs-dog-2"
predictor = TensorFlowPredictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker.Session())

In [ ]:
result = predictor.predict(data)
print(result)